In [121]:
using LinearAlgebra

# Compute the largest eigenvalue and its corresponding eigenvector of a square matrix `A` using the power method.
"""
# Arguments
- `A`: A square matrix with real or complex entries.
- `num_iterations`: Maximum number of iterations (default: 1000).
- `tolerance`: Convergence threshold for eigenvalue change (default: 1e-8).
- `round_digits`: Number of decimal places to round the results (default: 4).

# Returns
A tuple `(eigenvalue, eigenvector)` where:
- `eigenvalue`: The leading eigenvalue of `A`.
- `eigenvector`: The corresponding normalized eigenvector.
"""
function power_method_eigen(A::AbstractMatrix{T}; num_iterations::Int=1000, tolerance::Float64=1e-8, round_digits::Int=4) where T<:Number
    # Ensure that A is square
    n, m = size(A)
    if n != m
        error("Matrix A must be square.")
    end

    # Handle the special case for zero matrix
    if all(A .== 0)
        # Return the first standard basis vector
        leading_eigenvector = zeros(m)
        leading_eigenvector[1] = 1
        return (0, leading_eigenvector)  # Eigenvalue for a zero matrix is always 0
    end

    # Initialize vector
   
    eigenvector = rand(Float64, m) .+ im * rand(Float64, m)  # Complex initialization for complex matrices
 

    # Normalize the vector
    eigenvector /= norm(eigenvector)

    eigenval_old = 0
    eigenval = 0

    for i in 1:num_iterations
        # Compute new vector
        eigenvector_new = A * eigenvector


        # Compute the Rayleigh quotient as an estimate of the eigenvalue
        eigenval = dot(eigenvector_new, eigenvector) / dot(eigenvector, eigenvector) 

        # # Normalize new vector
        eigenvector_new /= norm(eigenvector_new)
 

        # Check for convergence
        if abs(eigenval - eigenval_old) < tolerance
            break
        end

        # Update eigenvector and eigenvalue
        eigenvector = eigenvector_new
        eigenval_old = eigenval
    end

    # Rounding
    largest_idx = argmax(abs.(eigenvector))
    eigenvector /= eigenvector[largest_idx]  # Scale so largest component is real

    eigenval = round(eigenval, digits=round_digits)
    eigenvector = round.(eigenvector, digits=round_digits)

    return eigenval, eigenvector
end


power_method_eigen

In [123]:

# Compute the largest singular value and its corresponding singular vectors of a matrix `A` using the power method.
"""
# Arguments
- `A`: A matrix with real or complex entries.
- `num_iterations`: Maximum number of iterations to perform (default: 1000).
- `tolerance`: Convergence threshold for singular value change (default: 1e-6).
- `round_digits`: Number of decimal places to round the results (default: 4).

# Returns
A tuple `(singular_value, left_singular_vector, right_singular_vector)` where:
- `singular_value`: The largest singular value of `A`.
- `left_singular_vector`: The corresponding left singular vector of `A`.
- `right_singular_vector`: The corresponding right singular vector of `A`.
"""
function power_method_singular(A::AbstractMatrix{T}; num_iterations::Int=1000, tolerance::Float64=1e-6, round_digits::Int=4) where T<:Number
    n, m = size(A)
    # Initialize a random vector
    x = randn(Float64, m) + im * randn(Float64, m)
    x /= norm(x)

    # Handle the special case for zero matrix
    if all(A .== 0)
        # Return zero singular value and standard basis vectors
        leading_vector = zeros(n)
        leading_vector[1] = 1
        return (0, leading_vector, leading_vector)
    end

    singularval_old = 0
    singularval = 0

    # Calculate A^H * A
    A_transposed_times_A = transpose(A) * A

    for i in 1:num_iterations
        x_new = A_transposed_times_A * x

        # Normalize x_new to get the new right singular vector approximation
        v = x_new / norm(x_new)

        # Compute corresponding singular value approximation
        singularval = norm(A * v)

        # Compute left singular vector approximation
        u = A * v / singularval

        # Check for convergence
        if abs(singularval - singularval_old) < tolerance
            break
        end

        # Normalize x for the next iteration
        x = x_new / norm(x_new)
        singularval_old = singularval
    end

    # Rounding
    singularval = round(singularval, digits=round_digits)
    u = round.(u, digits=round_digits)
    v = round.(v, digits=round_digits)

    return singularval, u, v
end




power_method_singular

In [125]:


# TESTING 
# Define a complex matrix
A = [2+3im 4-1im ; 4-1im 5+2im]
 

# Compute the largest eigenvalue and eigenvector
eigenvalue, eigenvector = power_method_eigen(A)
eigenvector = eigenvector / norm(eigenvector)
println("Eigenvalue: ", eigenvalue)
println("Eigenvector: ", round.(eigenvector, digits=4))

 

println();

# Compute eigenvalues and eigenvectors
println("EXPECTED RESULTS")
eigen_result = eigen(A)

# Find the index of the eigenvalue with the largest magnitude
largest_index = argmax(abs.(eigen_result.values)) # Use broadcasting with abs

# Retrieve the largest eigenvalue
largest_eigenvalue = eigen_result.values[largest_index]

# Print the largest eigenvalue
println("Eigenvalue: ", largest_eigenvalue)

# Retrieve and print the corresponding eigenvector
v1 = eigen_result.vectors[:, largest_index]
v1 = v1 / norm(v1)
println("Eigenvector: ", v1)





println();
println();

 

# Compute the largest singular value and singular vectors using your function
singular_value, u, v = power_method_singular(A)
u = u / norm(u)
v = v / norm(v)

println("Singular Value (Custom): ", singular_value)
println("Left Singular Vector (Custom, u): ", u)
println("Right Singular Vector (Custom, v): ", v)

println("\nEXPECTED RESULTS")
# Perform SVD using Julia's built-in function
svd_result = svd(A)

# Extract the largest singular value and corresponding singular vectors
singular_value_builtin = svd_result.S[1]
u_builtin = svd_result.U[:, 1]
v_builtin = svd_result.V[:, 1]

# Normalize Julia's singular vectors
u_builtin = u_builtin / norm(u_builtin)
v_builtin = v_builtin / norm(v_builtin)

println("Singular Value (Built-in): ", singular_value_builtin)
println("Left Singular Vector (Built-in, u_builtin): ", u_builtin)
println("Right Singular Vector (Built-in, v_builtin): ", v_builtin)

# Compute phase differences
phase_u = angle(dot(u_builtin, u))
phase_v = angle(dot(v_builtin, v))

# Adjust your singular vectors to align phases
u_adjusted = u * exp(-im * phase_u)
v_adjusted = v * exp(-im * phase_v)

# Compare the adjusted singular vectors with Julia's
u_difference = norm(u_adjusted - u_builtin)
v_difference = norm(v_adjusted - v_builtin)

println("\nAfter adjusting for phase difference:")
println("Difference between left singular vectors: ", u_difference)
println("Difference between right singular vectors: ", v_difference)
 

Eigenvalue: 7.7705 - 1.3877im
Eigenvector: ComplexF64[0.5667 + 0.0156im, 0.8238 + 0.0im]

EXPECTED RESULTS
Eigenvalue: 7.770500093323408 + 1.3877180901069988im
Eigenvector: ComplexF64[0.5666832729702826 + 0.015573114238585567im, 0.82378853248185 + 0.0im]


Singular Value (Custom): 7.8934
Left Singular Vector (Custom, u): ComplexF64[0.05259986692250504 + 0.5644985718204201im, 0.09889974978394958 + 0.817797930973852im]
Right Singular Vector (Custom, v): ComplexF64[0.15099979690540974 + 0.5463992650934827im, 0.2411996755866545 + 0.7876989405456375im]

EXPECTED RESULTS
Singular Value (Built-in): 7.897601864737666
Left Singular Vector (Built-in, u_builtin): ComplexF64[-0.5636101762178798 - 0.13498733752173517im, -0.8000201085444589 - 0.1552089362664275im]
Right Singular Vector (Built-in, v_builtin): ComplexF64[-0.5795498356720985 - 0.0im, -0.8141676286968051 - 0.03539859255203497im]

After adjusting for phase difference:
Difference between left singular vectors: 0.017124763569591323
Differe